<a href="https://colab.research.google.com/github/Hideyuki-Machida/ML_demos/blob/main/Movie_Recommender_Qdrant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 映画推薦システム

このノートブックは、こちらの内容と Google Colab Notebook をPineconeではなくQdrantに置き換えたものです。

* [Pinecone - Movie Recommender](https://docs.pinecone.io/docs/movie-recommender)
* [Colab](https://colab.research.google.com/github/pinecone-io/examples/blob/master/recommendation/movie-recommender/00_movie_recommender.ipynb)

このノートブックでは、類似検索が、シンプルな映画レコメンダーシステムを構築するのに役立つことを説明します。このレコメンダーシステムには3つのパートがあります：

- 映画の評価を含むデータセット
- 映画とユーザーを埋め込むための2つのディープラーニングモデル
- それらの埋め込みに対して類似性検索を行うためのベクトルインデックス

我々の推薦システムのアーキテクチャを以下に示す。ユーザモデルと映画モデルの2つのモデルがあり、ユーザと映画の埋め込みを生成する。2つのモデルは、多次元ベクトル空間におけるユーザと映画の近接度が、ユーザがその映画に対して与えた評価に依存するように学習される。つまり、ユーザが映画に高評価を与えた場合、その映画は多次元ベクトル空間においてユーザに近くなり、逆もまた同様である。これにより、最終的に映画の嗜好が似ているユーザーと、そのユーザーが高評価を与えた映画がベクトル空間上で近づくことになる。このベクトル空間におけるユーザーの類似性検索は、他のユーザーと共有された映画の嗜好に基づいて新しい推薦を与えることになる。


<center><div> <img src="https://raw.githubusercontent.com/pinecone-io/examples/master/movie_recommender/assets/movie-recommender.png" alt="Drawing" style="width:300px;"/></div> </center>

## 依存関係のインストール

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.1 MB/s eta 0:00:00


## データセットを読み込む

このプロジェクトでは、[MovieLens 25M Dataset]("https://grouplens.org/datasets/movielens/25m/") のサブセットを使用します。このデータセットには、[MovieLens 25M Dataset]("https://grouplens.org/datasets/movielens/25m/")の最新作〜1万本の映画について、3万人以上のユニークユーザーが提供した〜100万件のユーザー評価が含まれています。このサブセットは、HuggingFaceデータセットで[こちら]("https://huggingface.co/datasets/pinecone/movielens-recent-ratings")から入手できます。

In [2]:
from datasets import load_dataset

# データセットをpandasのdatafameにロードする
movies = load_dataset("pinecone/movielens-recent-ratings", split="train").to_pandas()

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10269 [00:00<?, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [3]:
# ユニークな映画だけを返すために重複を削除する。
unique_movies = movies.drop_duplicates(subset="imdb_id")
unique_movies

,imdb_id,movie_id,user_id,rating,title,poster
0,tt5027774,6705,4556,4.0,"Three Billboards Outside Ebbing, Missouri (2017)",https://m.media-amazon.com/images/M/MV5BMjI0OD...
1,tt5463162,7966,20798,3.5,Deadpool 2 (2018),https://m.media-amazon.com/images/M/MV5BMDkzNm...
2,tt4007502,1614,26543,4.5,Frozen Fever (2015),https://m.media-amazon.com/images/M/MV5BMjY3YT...
3,tt4209788,7022,4106,4.0,Molly's Game (2017),https://m.media-amazon.com/images/M/MV5BNTkzMz...
4,tt2948356,3571,15259,4.0,Zootopia (2016),https://m.media-amazon.com/images/M/MV5BOTMyMj...
...,...,...,...,...,...,...
782341,tt6148782,7522,24649,3.5,King of Peking (2017),https://m.media-amazon.com/images/M/MV5BNjQzOT...
794948,tt3620120,9477,27243,5.0,Candiland (2016),https://m.media-amazon.com/images/M/MV5BMTQ5MT...
796912,tt2404567,3915,30566,1.0,From Bedrooms to Billions (2014),https://m.media-amazon.com/images/M/MV5BMjAzMT...
802681,tt8809652,10216,1463,2.5,10 Minutes Gone (2019),https://m.media-amazon.com/images/M/MV5BZDdiYj...


## 埋め込みモデルの初期化

`user_model`と`movie_model`はTensorflow Kerasを用いて学習される。`user_model`は与えられた`user_id`を映画と同じベクトル空間における32次元の埋め込みに変換し、ユーザの映画の好みを表現する。そして、多次元空間におけるユーザーの位置への近さに基づいて、おすすめの映画がフェッチされる。

同様に、`movie_model`は与えられた`movie_id`を他の類似した映画と同じベクトル空間内の32次元の埋め込みに変換する。

In [4]:
from huggingface_hub import from_pretrained_keras

# ユーザーモデルとムービーモデルをhuggingfaceからロードする。
user_model = from_pretrained_keras("pinecone/movie-recommender-user-model")
movie_model = from_pretrained_keras("pinecone/movie-recommender-movie-model")

config.json not found in HuggingFace Hub.


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

config.json not found in HuggingFace Hub.


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

## ムービー埋め込みを作成する

事前に学習された `movie_model` を使って、ムービーの埋め込みを作成します。すべてのムービー埋め込みはQdrantの新しいインデックス `"movie-emb"` にアップサートされます。

In [5]:
from tqdm.auto import tqdm

# 64バッチを使用する
batch_size = 64
data_index = []

for i in tqdm(range(0, len(unique_movies), batch_size)):
    # バッチの終わりを見つける
    i_end = min(i+batch_size, len(unique_movies))
    # バッチを取り出す
    batch = unique_movies.iloc[i:i_end]
    # バッチに対するエンベッディングを生成
    emb = movie_model.predict(batch['movie_id']).tolist()
    # メタデータを取得
    meta = batch.to_dict(orient='records')
    # IDを作成
    ids = batch["imdb_id"].values.tolist()
    # すべてをupsertリストに追加
    to_upsert = list(zip(ids, emb, meta))
    data_index = data_index + to_upsert


  0%|          | 0/161 [00:00<?, ?it/s]

1/1 [==============================] - 0s 44ms/step


## Qdrantを設定

Qdrantにムービーの埋め込みができました。レコメンデーションを取得するには、2つの方法があります：

1. ユーザ埋め込みモデルと `user_id`s を使ってユーザの埋め込みを取得し、最も似ている映画の埋め込みを取得する。
2. 既存のムービーエンベッディングを使って、他の類似ムービーを検索する。

どちらも同じアプローチを使いますが、唯一の違いは、データソース（ユーザー対ムービー）と埋め込みモデル（ユーザー対ムービー）です。

タスク **1** から始めます。

In [6]:
!pip install qdrant-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [7]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(path="./db")

In [8]:
from qdrant_client import models

qdrant_client.recreate_collection(
    collection_name="movie-lens",
    vectors_config=models.VectorParams(
        size=len(data_index[0][1][0]), distance=models.Distance.COSINE
    ),
    on_disk_payload=True,
)

True

In [9]:
from typing import List, Any
from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client.http.models import PointStruct

def make_chunks(lst: List[Any], n=100) -> List[List[Any]]:
    n = max(1, n)
    return (lst[i : i + n] for i in range(0, len(lst), n))

points = []

for i in tqdm(range(0, len(data_index))):
    id = data_index[i][0]
    vector = data_index[i][1][0]
    payload = data_index[i][2]
    point = PointStruct(id=i, vector=vector, payload=payload)
    points.append(point)

points_list = make_chunks(points, n=100)


  0%|          | 0/10269 [00:00<?, ?it/s]

In [10]:
for points in points_list:
    operation_info = qdrant_client.upsert(
        collection_name="movie-lens",
        points=points
    )
    if operation_info.status.value != "completed":
        print(f"Failed: {operation_info.status.value}")

In [11]:
qdrant_client.get_collection(collection_name="movie-lens")

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=10269, indexed_vectors_count=0, points_count=10269, segments_count=1, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=32, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=None, replication_factor=None, write_consistency_factor=None, on_disk_payload=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=None, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), payload_schema={})

## おすすめをゲット

私たちは今、映画の埋め込みをQdrantに保存しています。おすすめを取得するためには、以下の2つの方法があります：

1. ユーザーの埋め込みモデルと私たちの `user_id` を使ってユーザーの埋め込みを取得し、それと最も類似した映画の埋め込みを取得します。
2. 既存の映画の埋め込みを使用して、他の類似した映画を取得します。
これらの両方は同じアプローチを使用していますが、唯一の違いはデータの源 (ユーザー対映画) と埋め込みモデル (ユーザー対映画) です。

まずはタスク **1** から始めます。

In [12]:
from IPython.core.display import HTML

def top_movies_user_rated(user):
    # ユーザーが評価した映画のリストを得る
    user_movies = movies[movies["user_id"] == user]
    # 評価の高い順に並べる
    top_rated = user_movies.sort_values(by=['rating'], ascending=False)
    # トップ14の映画を返す
    return top_rated['poster'].tolist()[:14], top_rated['rating'].tolist()[:14]

def display_posters(posters):
    figures = []
    for poster in posters:
        figures.append(f'''
            <figure style="margin: 5px !important;">
              <img src="{poster}" style="width: 120px; height: 150px" >
            </figure>
        ''')
    return HTML(data=f'''
        <div style="display: flex; flex-flow: row wrap; text-align: center;">
        {''.join(figures)}
        </div>
    ''')

ユーザー`3`の最高評価の映画を見てみましょう:

In [13]:
user = 3
top_rated, scores = top_movies_user_rated(user)
display_posters(top_rated)

In [14]:
print(scores)

[4.5, 4.0, 4.0, 2.5, 2.5]


ユーザー`3`は、これらの 5 つの映画を評価し、*Big Hero 6*、*シビル ウォー*、*アベンジャーズ* に高得点を付けました。彼らは、*Arrival* や *The Martian* などの SF 映画にはそれほど熱心ではないようです。

次に、このユーザーにおすすめの映画を作成する方法を見てみましょう。

`get_recommendations`関数を定義することから始めます。特定の`user_id`を指定すると、これは`user_model`を使用してユーザー埋め込み (`xq`) を作成します。次に、Qdrant (`xc`) から最も類似した映画ベクトルを取得し、関連する映画ポスターを抽出して、後で表示できるようにします。

In [15]:
def get_recommendations(user):
    # ユーザーの埋め込みを生成する
    xq = user_model([user]).numpy().tolist()
    # ユーザと映画のベクトル間の余弦類似度を計算し、トップkの映画を返す

    xc = qdrant_client.search(
        collection_name="movie-lens",
        query_vector=xq,
        limit=14,
    )

    result = []
    # 結果を繰り返し、映画ポスターを抽出する
    for match in xc:
        poster = match.payload['poster']
        result.append(poster)
    return result

## ユーザーへのおすすめ

In [16]:
urls = get_recommendations(user)
display_posters(urls)


これは良さそうです。上位の結果は、ユーザーの`3`つのお気に入りの結果と実際に一致しています。これに続いて、マーベルのスーパーヒーロー映画がたくさん見られますが、現在の評価から判断すると、ユーザー「3」がおそらく楽しめるでしょう。

別のユーザーを見てみましょう。今回は`128`を選択します。

In [17]:
user = 128
top_rated, scores = top_movies_user_rated(user)
display_posters(top_rated)

In [18]:
print(scores)

[4.5, 4.5, 4.5, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0]


このユーザーは何でも好きなようで、いろいろなものをミックスして薦められたりもする...。

In [19]:
urls = get_recommendations(user)
display_posters(urls)


これは良さそうです。上位の結果は、ユーザーの`3`つのお気に入りの結果と実際に一致しています。これに続いて、マーベルのスーパーヒーロー映画がたくさん見られますが、現在の評価から判断すると、ユーザー「3」がおそらく楽しめるでしょう。

別のユーザーを見てみましょう。今回は`128`を選択します。

In [20]:
user = 128
top_rated, scores = top_movies_user_rated(user)
display_posters(top_rated)

このユーザーはすべてが気に入っているように見えるため、さまざまなものの組み合わせも推奨されます...

In [21]:
urls = get_recommendations(user)
display_posters(urls)

In [22]:
user = 20000
top_rated, scores = top_movies_user_rated(user)
display_posters(top_rated)

In [23]:
print(scores)

[5.0, 4.0, 3.5, 3.5, 3.5, 3.0, 1.0]


このユーザーにはアクション映画への傾向がさらに見られるため、同様のアクションに焦点を当てたレコメンデーションが表示されることが期待されます。

In [24]:
urls = get_recommendations(user)
display_posters(urls)

## 類似の映画を探す


次に、類似した映画を見つける方法を見てみましょう。

まず、`get_similar_movies` 関数を定義します。特定の `imdb_id` を指定すると、Qdrant に保存されているその ID の既存の埋め込みを使用して直接クエリを実行します。

In [25]:
# 松ぼっくりインデックスで類似映画を検索する
def get_similar_movies(imdb_id):
    # 映画と埋め込みベクトル間の余弦類似度を計算し、トップkの映画を返す
    xc = index.query(id=imdb_id, top_k=14, include_metadata=True)
    result = []
    # 結果を繰り返し、映画のポスターを抽出する
    for match in xc['matches']:
        poster = match['metadata']['poster']
        result.append(poster)
    return result

In [26]:
# アベンジャーズ／インフィニティ・ウォー』
imdb_id = "tt4154756"
# unique_moviesからimdbidをフィルタリングする
movie = unique_movies[unique_movies["imdb_id"] == imdb_id]
movie

,imdb_id,movie_id,user_id,rating,title,poster
11,tt4154756,1263,153,4.0,Avengers: Infinity War - Part I (2018),https://m.media-amazon.com/images/M/MV5BMjMxNj...


In [27]:
# 映画ポスターの掲示
display_posters(movie["poster"])

今、私たちは*アベンジャーズを持っています： インフィニティ・ウォー』*。この映画に似ている映画を探してみましょう。

In [28]:
unique_movies_ids = unique_movies.reset_index()[["index"]]
unique_movies_ids[unique_movies_ids["index"] == movie.index[0]].index[0]
unique_movies_ids

,index
0,0
1,1
2,2
3,3
4,4
...,...
10264,782341
10265,794948
10266,796912
10267,802681


In [29]:
# 松ぼっくりインデックスで類似映画を検索する
def get_similar_movies(imdb_id):
    # 映画と埋め込みベクトル間の余弦類似度を計算し、上位k個の映画を返す
    ids = [ imdb_id ]
    xc = qdrant_client.recommend(
        positive=ids,
        collection_name="movie-lens",
        with_payload=True,
        with_vectors=False,
        limit=14
    )

    result = []
    # 結果を繰り返し、映画ポスターを抽出する
    for match in xc:
        poster = match.payload['poster']
        result.append(poster)
    return result

id = int(unique_movies_ids[unique_movies_ids["index"] == movie.index[0]].index[0])
similar_movies = get_similar_movies(id)
display_posters(similar_movies)

上位の結果は*アベンジャーズと密接に一致している： インフィニティ・ウォー*に最も似ている映画のトップは、映画そのものである。これに続いて、他のマーベル・スーパーヒーロー映画が多く見られる。


別の映画を見てみよう。今度はアニメだ。

In [30]:
# モアナのimdbid
imdb_id = "tt3521164"
# unique_movies から imdbid をフィルタリングする
movie = unique_movies[unique_movies["imdb_id"] == imdb_id]
movie

,imdb_id,movie_id,user_id,rating,title,poster
97,tt3521164,5138,24875,5.0,Moana (2016),https://m.media-amazon.com/images/M/MV5BMjI4Mz...


In [31]:
# 映画のポスターを飾る
display_posters(movie["poster"])

In [32]:
id = int(unique_movies_ids[unique_movies_ids["index"] == movie.index[0]].index[0])
similar_movies = get_similar_movies(id)
display_posters(similar_movies)

この結果の質がまたいい。上位の結果はたくさんのアニメを返している。